In [178]:
import pandas as pd
import numpy as np

In [179]:
# download best model
import wandb
api = wandb.Api()
run = api.run("deep_learning_hsa/Abgabe_02/nft17upj")
model=run.file("model-best.h5").download(replace=True)

In [180]:
from sharedtask02.Model.model import create_traindataset
from sklearn.preprocessing import StandardScaler
# Import Data
from sklearn.preprocessing import MinMaxScaler
# Read in the csv data using pandas
train = pd.read_csv('../../data/01_train/train.csv', header=None)

sampleTest = pd.read_csv('../../data/01_train/sampleSubmission.csv', header=None)
sampleTest=sampleTest.to_numpy()

test_original = pd.read_csv('../../data/01_train/sampleTest.csv', header=None)
test_original=test_original.to_numpy()
test=test_original.copy()

In [181]:
x_train, y_train, x_test, y_test = create_traindataset(data=train)
scaler = StandardScaler()
#scal Data

scaler.fit_transform(x_train.reshape(-1, 1))

shape = x_train.shape
x_train = scaler.transform(x_train.reshape(-1, 1))
x_train = x_train.reshape(shape)

shape = test.shape
test = scaler.transform(test.reshape(-1, 1))
test = test.reshape(shape)

print("Data successful preprocessed!")

Data successful preprocessed!


In [182]:
#test if valide
test_reinitalisiert=scaler.inverse_transform(test)

if np.array_equal(test_original,test_reinitalisiert):
    print('Erfolg')
else:
    print('Rundungsfehler')
    for r in range(test_original.shape[0]):
        for c in range(test_original.shape[1]):
            if test_original[r, c] != round(test_reinitalisiert[r, c],10):
                print(r,c,test_original[r,c],test_reinitalisiert[r,c])


Rundungsfehler


In [183]:
import keras

model = keras.models.load_model('model-best.h5')

In [184]:
import tensorflow as tf
test = np.expand_dims(np.array(test), axis=-1)
test = tf.convert_to_tensor(test)
pred_sample = model.predict(x=test)
pred_sample_ausgabe=np.ndarray.tolist(pred_sample)


1/1 [==============================] - 0s 107ms/step


In [185]:
from sklearn.metrics import mean_squared_error

#pred_test = model.predict(x=x_train)
print(mean_squared_error(pred_sample, sampleTest))

pred_train = model.predict(x=x_train)
y_train =    y_train.reshape(pred_train.shape)
#pred_train = np.expand_dims(np.array(pred_train), axis=-1)
print(mean_squared_error(pred_train, y_train))

pred_test = model.predict(x=x_test)
y_test =    y_test.reshape(pred_test.shape)
print(mean_squared_error(pred_test, y_test))


ValueError: y_true and y_pred have different number of output (1!=7)

In [ ]:
pred_test-y_test

In [ ]:
# predict Data

#pred_sampleTest= test.append(pred_sampleTest)
np.savetxt("../../data/01_train/pred_sample.csv",
           pred_sample_ausgabe,
           delimiter =", ",
           fmt ='% s')

print("pred_test.csv successful saved!")